In [1]:
!pip install streamlit==1.28.0
!pip install scikit-learn==1.3.2
!pip install joblib==1.3.2
!pip install numpy==1.26.0
!pip install pandas==2.1.1


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('predictive_maintenance.csv')

In [4]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


In [6]:
df['Type'].nunique()

3

In [7]:
df.isnull().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64

#Some Understandings : <br>
1. First we need to transform some columns by ordinal encoding/ label encoding<br>
2. There are no missing values<br>
3. `Type` Column has 3 types - L/M/H which is ordinal data , so apply ordinal encoding here<br>
The target variable here is `Target` only, i.e. to predict failure or not
4. Notice that, there are two output target variables `Target` and `Failure Type`<br> So prediect here multiclass Failure Type Variable<br>

5. This is a classification problem <br>

In [8]:
columns = df.columns.tolist()
columns

['UDI',
 'Product ID',
 'Type',
 'Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Target',
 'Failure Type']

In [9]:
df[df['Target']==1]

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
50,51,L47230,L,298.9,309.1,2861,4.6,143,1,Power Failure
69,70,L47249,L,298.9,309.0,1410,65.7,191,1,Power Failure
77,78,L47257,L,298.8,308.9,1455,41.3,208,1,Tool Wear Failure
160,161,L47340,L,298.4,308.2,1282,60.7,216,1,Overstrain Failure
161,162,L47341,L,298.3,308.1,1412,52.3,218,1,Overstrain Failure
...,...,...,...,...,...,...,...,...,...,...
9758,9759,L56938,L,298.6,309.8,2271,16.2,218,1,Tool Wear Failure
9764,9765,L56944,L,298.5,309.5,1294,66.7,12,1,Power Failure
9822,9823,L57002,L,298.5,309.4,1360,60.9,187,1,Overstrain Failure
9830,9831,L57010,L,298.3,309.3,1337,56.1,206,1,Overstrain Failure


In [10]:

from sklearn.model_selection import train_test_split

#X = df.iloc[:,2:8]
#y = df.iloc[:,-1]
features = [
    "Type",
    "Air temperature [K]",
    "Process temperature [K]",
    "Rotational speed [rpm]",
    "Torque [Nm]",
    "Tool wear [min]"
]

X = df[features]
y = df["Failure Type"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [11]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(categories=[['L', 'M', 'H']])

# Encode Type column: L → 0, M → 1, H → 2
#X_train.loc[:, 'Type'] = oe.fit_transform(X_train[['Type']]).astype(int)
#X_test.loc[:, 'Type'] = oe.transform(X_test[['Type']]).astype(int)

X_train[['Type']] = oe.fit_transform(X_train[['Type']])
X_test[['Type']] = oe.transform(X_test[['Type']])

In [12]:
X_train.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
7755,1.0,300.4,311.8,1362,47.6,188
2671,1.0,299.7,309.3,1399,41.9,221
4377,1.0,301.8,309.6,1436,49.0,214
3137,0.0,300.4,309.9,1567,36.5,72
1935,1.0,298.1,307.9,1377,55.1,42


In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [14]:
y_train.shape

(8000,)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

log_reg = LogisticRegression(solver='lbfgs', max_iter=10000)
log_reg.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = log_reg.predict(X_test)

log_reg_train = round(log_reg.score(X_train, y_train) * 100, 2)
log_reg_accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)


print("Training Accuracy    :",log_reg_train ,"%")
print("Model Accuracy Score :",log_reg_accuracy ,"%")
print("\033[1m--------------------------------------------------------\033[0m")
print("Classification_Report: \n",classification_report(y_test,y_pred))
print("\033[1m--------------------------------------------------------\033[0m")


Training Accuracy    : 98.16 %
Model Accuracy Score : 98.0 %
--------------------------------------------------------
Classification_Report: 
               precision    recall  f1-score   support

           0       0.74      0.64      0.68        22
           1       0.98      1.00      0.99      1930
           2       0.82      0.56      0.67        16
           3       0.88      0.74      0.80        19
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         9

    accuracy                           0.98      2000
   macro avg       0.57      0.49      0.52      2000
weighted avg       0.97      0.98      0.98      2000

--------------------------------------------------------


c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [17]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

decision_tree_train = round(decision_tree.score(X_train, y_train) * 100, 2)
decision_tree_accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)

print("Training Accuracy    :",decision_tree_train ,"%")
print("Model Accuracy Score :",decision_tree_accuracy ,"%")
print("\033[1m--------------------------------------------------------\033[0m")
print("Classification_Report: \n",classification_report(y_test,y_pred))
print("\033[1m--------------------------------------------------------\033[0m")


Training Accuracy    : 100.0 %
Model Accuracy Score : 97.95 %
--------------------------------------------------------
Classification_Report: 
               precision    recall  f1-score   support

           0       0.95      0.82      0.88        22
           1       0.99      0.99      0.99      1930
           2       0.60      0.56      0.58        16
           3       0.94      0.84      0.89        19
           4       0.00      0.00      0.00         4
           5       0.14      0.11      0.12         9

    accuracy                           0.98      2000
   macro avg       0.60      0.55      0.58      2000
weighted avg       0.98      0.98      0.98      2000

--------------------------------------------------------


In [18]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

train_acc = random_forest.score(X_train, y_train)
test_acc = accuracy_score(y_test, y_pred)

print("Training Accuracy    :", round(train_acc * 100, 2), "%")
print("Model Accuracy Score :", round(test_acc * 100, 2), "%")
#print("\033[1m--------------------------------------------------------\033[0m")
#print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\033[1m--------------------------------------------------------\033[0m")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\033[1m--------------------------------------------------------\033[0m")

Training Accuracy    : 100.0 %
Model Accuracy Score : 98.1 %
--------------------------------------------------------

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.68      0.81        22
           1       0.98      1.00      0.99      1930
           2       1.00      0.56      0.72        16
           3       1.00      0.42      0.59        19
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         9

    accuracy                           0.98      2000
   macro avg       0.66      0.44      0.52      2000
weighted avg       0.97      0.98      0.98      2000

--------------------------------------------------------


c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [19]:
# Support Vector Machines
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

svc_train = round(svc.score(X_train, y_train) * 100, 2)
svc_accuracy = round(accuracy_score(y_pred_svc, y_test) * 100, 2)

print("Training Accuracy    :",svc_train ,"%")
print("Model Accuracy Score :",svc_accuracy ,"%")
print("\033[1m--------------------------------------------------------\033[0m")
print("Classification_Report: \n",classification_report(y_test,y_pred_svc))
print("\033[1m--------------------------------------------------------\033[0m")

Training Accuracy    : 96.72 %
Model Accuracy Score : 96.55 %
--------------------------------------------------------
Classification_Report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.97      1.00      0.98      1930
           2       0.00      0.00      0.00        16
           3       0.67      0.11      0.18        19
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         9

    accuracy                           0.97      2000
   macro avg       0.27      0.18      0.19      2000
weighted avg       0.94      0.97      0.95      2000

--------------------------------------------------------


c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [26]:
import pickle

#Save as pickle
pickle.dump(random_forest, open('model.pkl','wb'))
pickle.dump(oe, open('type_encoder.pkl', 'wb'))
pickle.dump(le, open('label_encoder.pkl', 'wb'))

# Load pickle files
random_forest = pickle.load(open("model.pkl", "rb"))
oe = pickle.load(open('type_encoder.pkl', 'rb'))
le = pickle.load(open('label_encoder.pkl', 'rb'))

print("\n✅ Pickle files saved successfully!")



✅ Pickle files saved successfully!


In [24]:
import joblib

#Save as joblib
joblib.dump(random_forest, 'model.joblib')
joblib.dump(oe, "type_encoder.joblib")
joblib.dump(le, "label_encoder.joblib")

#Load joblib files
random_forest = joblib.load('model.joblib')
oe = joblib.load('type_encoder.joblib')
le = joblib.load('label_encoder.joblib')

print("\n✅ Model and encoders saved successfully!")


✅ Model and encoders saved successfully!


In [1]:
# ===============================
# 1. IMPORT LIBRARIES
# ===============================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import joblib

# ===============================
# 2. LOAD DATASET
# ===============================
df = pd.read_csv("predictive_maintenance.csv")

# ===============================
# 3. FEATURES & TARGET
# ===============================
# Columns:
# 0 - UDI
# 1 - Product ID
# 2 - Type
# 3 - Air temperature [K]
# 4 - Process temperature [K]
# 5 - Rotational speed [rpm]
# 6 - Torque [Nm]
# 7 - Tool wear [min]
# Last column - Failure Type

#X = df.iloc[:, 2:8]
#y = df.iloc[:, -1]
features = [
    "Type",
    "Air temperature [K]",
    "Process temperature [K]",
    "Rotational speed [rpm]",
    "Torque [Nm]",
    "Tool wear [min]"
]

X = df[features]
y = df["Failure Type"]

# ===============================
# 4. TRAIN TEST SPLIT
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ===============================
# 5. ENCODE "Type" COLUMN (L, M, H)
# ===============================
oe = OrdinalEncoder(categories=[['L', 'M', 'H']])

X_train[['Type']] = oe.fit_transform(X_train[['Type']])
X_test[['Type']] = oe.transform(X_test[['Type']])

# ===============================
# 6. ENCODE TARGET VARIABLE
# ===============================
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# ===============================
# 7. RANDOM FOREST MODEL
# ===============================
random_forest = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

random_forest.fit(X_train, y_train)

# ===============================
# 8. PREDICTION & EVALUATION
# ===============================
y_pred = random_forest.predict(X_test)

train_acc = random_forest.score(X_train, y_train)
test_acc = accuracy_score(y_test, y_pred)

print("Training Accuracy    :", round(train_acc * 100, 2), "%")
print("Model Accuracy Score :", round(test_acc * 100, 2), "%")
print("\033[1m--------------------------------------------------------\033[0m")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\033[1m--------------------------------------------------------\033[0m")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\033[1m--------------------------------------------------------\033[0m")

# ===============================
# 9. SAVE MODEL & ENCODERS
# ===============================
joblib.dump(random_forest, "model.joblib")
joblib.dump(oe, "type_encoder.joblib")
joblib.dump(le, "label_encoder.joblib")

print("\n✅ Model and encoders saved successfully!")


Training Accuracy    : 100.0 %
Model Accuracy Score : 98.1 %
--------------------------------------------------------

Confusion Matrix:
 [[  15    7    0    0    0    0]
 [   0 1930    0    0    0    0]
 [   0    7    9    0    0    0]
 [   0   11    0    8    0    0]
 [   0    4    0    0    0    0]
 [   0    9    0    0    0    0]]
--------------------------------------------------------

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.68      0.81        22
           1       0.98      1.00      0.99      1930
           2       1.00      0.56      0.72        16
           3       1.00      0.42      0.59        19
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         9

    accuracy                           0.98      2000
   macro avg       0.66      0.44      0.52      2000
weighted avg       0.97      0.98      0.98      2000

--------------------------------------

c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Final Year Project\Source Code\my_app_env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu